## Swarm

This concept was initially designed by OpenAI ---> Handoff Condition

The Swarm implements a team in which agent can handoff task to another agent based on their capabilities.

It is a group chat where agents take turn to generate a response, participant agents broadcast their response so all the agents share the same message context.

In this, next agent is selected based on the most recent HandofMessage message in the context.

In the `AssistantAgent` we can set the handoffs arguments to specify which agents it can handoff to. This will only work if a swarm is selected.

The overall process is:
1. Each agent can generate a HandoffMessage to indicate which agents it can hand off to.
2. The initial agent(s) begin the task and decide locally whether to hand off and to whom.
3. When a HandoffMessage is generated, the designated agent takes over with the same context.
4. This process repeats until a termination condition is reached.

![alt text](swarm_customer_support.svg)

## Travel Refund Teams Workflow
1. Travel Agent initiates the conversation and try to understand the user's request.
2. Based on the request:
    - For generic info, it gives that and would ideally handoff to user.
    - For refund related requests, Agent handoff to the Flight refunder.
3. Flight refunder uses the tool **refund_flight** tool when required.
4. If agent hands off to **user**, team execution will stop and wait for user's response.
5. If user provides an input, it is sent back to team as a Handoff Message and **is directed to the agent that originally reuested users input**.
6. This process will continue until Travel agent determines the task is complete and terminate the workflow.

In [1]:
from typing import Any, Dict, List

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import HandoffTermination, TextMentionTermination
from autogen_agentchat.messages import HandoffMessage
from autogen_agentchat.teams import Swarm
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
def refund_flight(flight_PNR:str)->str:
    return f"Refunded flight with PNR {flight_PNR}"

model_client = OpenAIChatCompletionClient(model='gpt-4o-mini')

In [17]:
## travel agent

travel_agent = AssistantAgent(
    name='travel_agent',
    model_client=model_client,
    handoffs=['flights_refunder','user'],
    system_message='''You are a travel agent.
    The flights_refunder is in charge of refunding flights.
    If you need information from the user, you must first send your message, then you can handoff to the user.
    Use TERMINATE when the travel planning is complete.
    '''
)

In [18]:
## flight refunder

flights_refunder = AssistantAgent(
    name='flights_refunder',
    model_client=model_client,
    tools=[refund_flight],
    handoffs=['travel_agent','user'],
    system_message="""You are an agent specialized in refunding flights.
    You only need flight PNR numbers to refund a flight.
    You have the ability to refund a flight using the refund_flight tool.
    If you need information from the user, you must first send your message, then you can handoff to the user.
    When the transaction is complete, handoff to the travel agent to finalize.""",
)

In [19]:
termination = HandoffTermination(target='user') | TextMentionTermination('TERMINATE')
team = Swarm(
    participants=[travel_agent, flights_refunder],
    termination_condition=termination
)

In [20]:
## resetting the 
await team.reset()

In [21]:
task = 'I want refund of my flight'

async def run_stream():
    task_result = await Console(team.run_stream(task=task))
    last_message = task_result.messages[-1]
    
    while isinstance(last_message, HandoffMessage) and last_message.target =='user':
        user_message = input("User:")
        
        task_result = await Console(
            team.run_stream(task=HandoffMessage(source='user',target=last_message.source, content=user_message))
        )
        
        last_message = task_result.messages[-1]
    
    
await run_stream()

---------- TextMessage (user) ----------
I want refund of my flight
---------- TextMessage (travel_agent) ----------
I can help you with that! Could you please provide me with some information about your flight? Specifically, I'll need the flight number and the reason for the refund.
---------- ToolCallRequestEvent (travel_agent) ----------
[FunctionCall(id='call_3M38LIwP72YTkhLm0OENg3MI', arguments='{}', name='transfer_to_user')]
---------- ToolCallExecutionEvent (travel_agent) ----------
[FunctionExecutionResult(content='Transferred to user, adopting the role of user immediately.', name='transfer_to_user', call_id='call_3M38LIwP72YTkhLm0OENg3MI', is_error=False)]
---------- HandoffMessage (travel_agent) ----------
Transferred to user, adopting the role of user immediately.
---------- HandoffMessage (user) ----------
ABC1234
---------- TextMessage (travel_agent) ----------
Thank you for providing your flight number. Could you please let me know the reason for the refund?
---------- To